# Importaciones

In [2]:
# Importar librerías necesarias
import os
import glob
import random
import pandas as pd

# Establecer la semilla para reproducibilidad
random.seed(42)

# Definir la carpeta para guardar los CSV
csv_dir = "./results/csv_files"
os.makedirs(csv_dir, exist_ok=True)  # Crea la carpeta si no existe

# Directorio raíz de las imágenes reales
# (Carpeta que contiene subcarpetas "Bacterial", "COVID-19" y "Normal")
real_data_dir = "./data/covid-chest-xray"

# Directorio de las imágenes aumentadas
# (Carpeta que contiene subcarpetas "Bacterial", "COVID-19" y "Normal")
augmented_data_dir = "./data/data_augmentation"

# Lista de clases según la estructura de tus carpetas
clases = ["Bacterial", "COVID-19", "Normal"]

In [2]:
def obtener_rutas_imagenes(ruta_base, extension=("jpg", "jpeg", "png", "tiff")):
    rutas = []
    for ext in extension:
        rutas.extend(glob.glob(os.path.join(ruta_base, f"*.{ext}")))
    return rutas


# Crear un diccionario con las rutas de imágenes reales para cada clase
imagenes_reales = {}
for clase in clases:
    ruta_clase = os.path.join(real_data_dir, clase)
    imagenes_reales[clase] = obtener_rutas_imagenes(ruta_clase)
    print(f"{clase}: {len(imagenes_reales[clase])} imágenes reales")

Bacterial: 47 imágenes reales
COVID-19: 341 imágenes reales
Normal: 486 imágenes reales


In [3]:
# Definir porcentajes
porc_train = 0.70
porc_val = 0.15
porc_test = 0.15

# Diccionarios para almacenar las rutas de cada conjunto
train_real = {}
val_real = {}
test_real = {}

categories = ['COVID-19', 'No-COVID-19']

for category in categories:
    if category == 'COVID-19':
        imagenes = imagenes_reales[category].copy()
    else:
        imagenes = imagenes_reales['Bacterial'].copy() + imagenes_reales['Normal'].copy()
    random.shuffle(imagenes)

    total = len(imagenes)
    n_train = int(total * porc_train)
    n_val = int(total * porc_val)
    # El resto va a test, asegurando que se asignen todas las imágenes
    n_test = total - n_train - n_val

    train_real[category] = imagenes[:n_train]
    val_real[category] = imagenes[n_train : n_train + n_val]
    test_real[category] = imagenes[n_train + n_val :]

    print(
        f"{category} - Total: {total}, Train: {len(train_real[category])}, Val: {len(val_real[category])}, Test: {len(test_real[category])}"
    )

COVID-19 - Total: 341, Train: 238, Val: 51, Test: 52
No-COVID-19 - Total: 533, Train: 373, Val: 79, Test: 81


In [ ]:
def crear_registro(rutas_dict, conjunto_name):
    registros = []
    for clase, rutas in rutas_dict.items():
        for ruta in rutas:
            registros.append({"Conjunto": conjunto_name, "Clase": clase, "Ruta": ruta})
    return pd.DataFrame(registros)


df_train = crear_registro(train_real, "train")
df_val = crear_registro(val_real, "val")
df_test = crear_registro(test_real, "test")

# Concatenar y mostrar el registro completo
df_registro = pd.concat([df_train, df_val, df_test], ignore_index=True)
print(df_registro.head())
print(df_registro.groupby(["Conjunto", "Clase"]).size())

# Guardar el registro de imágenes reales
csv_registro_imagenes = os.path.join(csv_dir, "registro_imagenes_reales_binary_2.csv")
df_registro.to_csv(csv_registro_imagenes, index=False)
print(f"Registro de imágenes guardado en: {csv_registro_imagenes}")

  Conjunto     Clase                                               Ruta
0    train  COVID-19  ./data/covid-chest-xray/COVID-19/COVID-19-246.png
1    train  COVID-19  ./data/covid-chest-xray/COVID-19/COVID-19-338.jpg
2    train  COVID-19  ./data/covid-chest-xray/COVID-19/COVID-19-42.jpeg
3    train  COVID-19   ./data/covid-chest-xray/COVID-19/COVID-19-99.png
4    train  COVID-19  ./data/covid-chest-xray/COVID-19/COVID-19-263....
Conjunto  Clase      
test      COVID-19        52
          No-COVID-19     81
train     COVID-19       238
          No-COVID-19    373
val       COVID-19        51
          No-COVID-19     79
dtype: int64
Registro de imágenes guardado en: ./results/csv_files/registro_imagenes_reales_binary_2.csv


In [12]:
def obtener_rutas_limitadas(ruta_base, cantidad, extension=("jpg", "jpeg", "png", "tiff")):
    imagenes = obtener_rutas_imagenes(ruta_base, extension)
    return random.sample(imagenes, min(cantidad, len(imagenes)))

# Diccionario para almacenar rutas de imágenes aumentadas por clase
imagenes_aug = {}
for category in categories:   
    # Insertar imágenes adicionales según la clase
    if category == "COVID-19":
        imagenes_aug[category] = obtener_rutas_limitadas("./data/data_augmentation/COVID-19", 500)
    else:
        imagenes_aug[category] = obtener_rutas_limitadas("./data/data_augmentation/Bacterial", 250)
        imagenes_aug[category] += obtener_rutas_limitadas("./data/data_augmentation/Normal", 250)
    
    print(f"{category}: {len(imagenes_aug[category])} imágenes aumentadas")

COVID-19: 500 imágenes aumentadas
No-COVID-19: 500 imágenes aumentadas


In [13]:
# Diccionario para el conjunto de entrenamiento final (reales + aumentadas)
train_final = {}
for category in categories:
    reales = train_real[category]
    aumentadas = imagenes_aug.get(category, [])
    train_final[category] = reales + aumentadas
    print(f"{category} - Train final: {len(reales)} reales + {len(aumentadas)} aumentadas = {len(train_final[category])} imágenes")

df_train_final = crear_registro(train_final, "train_final")

COVID-19 - Train final: 238 reales + 500 aumentadas = 738 imágenes
No-COVID-19 - Train final: 373 reales + 500 aumentadas = 873 imágenes


In [14]:
# Mostrar resumen por clase
print(df_train_final.groupby("Clase").size())

# Guardar el registro del conjunto de entrenamiento final (reales + aumentadas)
csv_train_final = os.path.join(csv_dir, "registro_train_final_binary_2.csv")
df_train_final.to_csv(csv_train_final, index=False)
print(f"Registro del conjunto de entrenamiento final guardado en: {csv_train_final}")

Clase
COVID-19       738
No-COVID-19    873
dtype: int64
Registro del conjunto de entrenamiento final guardado en: ./results/csv_files/registro_train_final_binary_2.csv


In [19]:
# Resumen de las cantidades en cada conjunto
resumen = {
    "Conjunto": ["Train (reales)", "Val", "Test"],
    "COVID-19": [
        len(train_real["COVID-19"]),
        len(val_real["COVID-19"]),
        len(test_real["COVID-19"]),
    ],
    "No-COVID-19": [
        len(train_real["No-COVID-19"]),
        len(val_real["No-COVID-19"]),
        len(test_real["No-COVID-19"]),
    ],
}

df_resumen = pd.DataFrame(resumen)
print("División de imágenes reales:")
print(df_resumen)

# Resumen de imágenes en el entrenamiento final (reales)
resumen_train_final = {
    "Clase": categories,
    "Train_final": [len(train_final[category]) for category in categories],
}

df_resumen_train_final = pd.DataFrame(resumen_train_final)
print("\nConjunto de entrenamiento final (reales):")
print(df_resumen_train_final)

División de imágenes reales:
         Conjunto  COVID-19  No-COVID-19
0  Train (reales)       238          373
1             Val        51           79
2            Test        52           81

Conjunto de entrenamiento final (reales):
         Clase  Train_final
0     COVID-19          238
1  No-COVID-19          373


In [11]:
csv_train_final = os.path.join(csv_dir, "registro_train_final_binary_2.csv")
df_train_final = pd.read_csv(csv_train_final)

csv_registro_imagenes = os.path.join(csv_dir, "registro_imagenes_reales_binary_2.csv")
df_registro = pd.read_csv(csv_registro_imagenes)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Crear generador para el conjunto de entrenamiento (solo rescaling)
train_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Crear el generador de datos a partir del DataFrame de entrenamiento (imágenes reales + aumentadas)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train_final,  # DataFrame con las imágenes de entrenamiento
    x_col="Ruta",  # Columna con las rutas de las imágenes
    y_col="Clase",  # Columna con las etiquetas de clase
    target_size=(224, 224),  # Redimensionar las imágenes a 224x224
    batch_size=32,
    class_mode="categorical",  # Para clasificación multiclase
    shuffle=True,
)

print("Clases encontradas:", train_generator.class_indices)

# Crear generador para test (solo rescaling)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Filtrar el DataFrame para obtener solo las imágenes del conjunto "test"
df_test = df_registro[df_registro["Conjunto"] == "test"]

# Crear el generador de datos para el conjunto de test
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,  # DataFrame con las imágenes reales de test
    x_col="Ruta",  # Columna que contiene las rutas de las imágenes
    y_col="Clase",  # Columna con las etiquetas de clase
    target_size=(224, 224),  # Redimensiona las imágenes a 224x224
    batch_size=32,
    class_mode="categorical",  # Para clasificación multiclase
    shuffle=False,  # No se baraja para mantener el orden de test
)

print("Clases encontradas en test:", test_generator.class_indices)

Found 1611 validated image filenames belonging to 2 classes.
Clases encontradas: {'COVID-19': 0, 'No-COVID-19': 1}
Found 133 validated image filenames belonging to 2 classes.
Clases encontradas en test: {'COVID-19': 0, 'No-COVID-19': 1}


In [3]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Dropout,
    Flatten,
    Dense,
    BatchNormalization,
    Activation,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
import json

2025-03-05 23:05:41.220743: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 23:05:41.500904: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 23:05:41.846679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741212342.137470     517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741212342.270578     517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 23:05:42.950009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Definición del modelo

In [4]:
# =============================================================================
# Definir la función que construye el modelo de CNN, con hiperparámetros ajustables.
# =============================================================================
def build_cnn(hp):
    model = Sequential(
        [
            # Primera capa convolucional: se parametriza el número de filtros (entre 32 y 64, de 16 en 16)
            Conv2D(
                hp.Int("filters_1", 32, 64, step=16),
                (3, 3),
                activation=None,
                input_shape=(224, 224, 3),
            ),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_1", 0.2, 0.5, step=0.1)),
            
            # Segunda capa convolucional
            Conv2D(hp.Int("filters_2", 64, 128, step=32), (3, 3), activation=None),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_2", 0.2, 0.5, step=0.1)),
            
            # Tercera capa convolucional
            Conv2D(hp.Int("filters_3", 128, 256, step=64), (3, 3), activation=None),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_3", 0.3, 0.6, step=0.1)),
            
            # Capa de aplanamiento para pasar a las capas densas
            Flatten(),
            
            # Capa densa con unidades parametrizables (entre 64 y 256, de 64 en 64)
            Dense(hp.Int("dense_units", 64, 256, step=64), activation=None),
            BatchNormalization(),
            Activation("relu"),
            Dropout(hp.Float("dropout_dense", 0.3, 0.6, step=0.1)),
            
            # Capa de salida: 2 neuronas para 2 clases, con activación softmax
            Dense(2, activation="softmax"),
        ]
    )

    # Definir el optimizador: Adam con tasa de aprendizaje seleccionada entre tres opciones.
    optimizer = Adam(learning_rate=hp.Choice("learning_rate", [1e-4, 5e-4, 1e-3]))

    # Compilar el modelo usando entropía cruzada categórica (para clasificación multiclase)
    # Se añaden varias métricas para evaluar: accuracy, precision, recall y AUC.
    model.compile(
        loss="binary_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy", "precision", "recall", "AUC"],
    )

    return model

# Carga de datos

# Entrenamiento del modelo

In [ ]:
# =============================================================================
# Configurar el Random Search para buscar los mejores hiperparámetros
# =============================================================================
tuner = RandomSearch(
    build_cnn,  # Función que construye el modelo
    objective="val_accuracy",  # Métrica a optimizar (precisión en validación)
    max_trials=10,  # Número máximo de combinaciones a probar
    executions_per_trial=1,  # Número de ejecuciones por cada combinación
    directory="cnn_chest-xray_problem_binary",  # Directorio para guardar resultados del tuner
    project_name="cnn_chest-xray_problem_binary",  # Nombre del proyecto
)

# =============================================================================
# Definir Early Stopping para detener el entrenamiento si no mejora la precisión de validación
# =============================================================================
early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True
)

# =============================================================================
# Realizar la búsqueda de hiperparámetros usando los generadores de entrenamiento y validación.
# Se asume que 'train_generator' y 'test_generator' están definidos previamente.
# =============================================================================
tuner.search(
    train_generator,
    validation_data=test_generator,  # Aquí test_generator actúa como conjunto de validación
    epochs=25,
    callbacks=[early_stopping],
    verbose=1,
)

Reloading Tuner from cnn_chest-xray_problem_binary_2/cnn_chest-xray_problem_binary_2/tuner0.json


In [14]:
# Mostrar resumen de la búsqueda de hiperparámetros
tuner.results_summary()

Results summary
Results in cnn_chest-xray_problem_binary_2/cnn_chest-xray_problem_binary_2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
filters_1: 32
dropout_1: 0.30000000000000004
filters_2: 96
dropout_2: 0.2
filters_3: 128
dropout_3: 0.3
dense_units: 192
dropout_dense: 0.3
learning_rate: 0.001
Score: 0.9924812316894531

Trial 05 summary
Hyperparameters:
filters_1: 32
dropout_1: 0.30000000000000004
filters_2: 96
dropout_2: 0.4
filters_3: 256
dropout_3: 0.3
dense_units: 64
dropout_dense: 0.3
learning_rate: 0.001
Score: 0.969924807548523

Trial 00 summary
Hyperparameters:
filters_1: 64
dropout_1: 0.4
filters_2: 128
dropout_2: 0.4
filters_3: 256
dropout_3: 0.5
dense_units: 64
dropout_dense: 0.3
learning_rate: 0.001
Score: 0.9624060392379761

Trial 01 summary
Hyperparameters:
filters_1: 64
dropout_1: 0.2
filters_2: 64
dropout_2: 0.2
filters_3: 128
dropout_3: 0.3
dense_units: 64
dropout_dense: 0.5
learning_rate: 0.0001
Score: 0.3

In [15]:
# =============================================================================
# Obtener el mejor modelo y sus hiperparámetros encontrados
# =============================================================================
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]
print(
    f"Mejor modelo encontrado con filtros: {best_hps.get('filters_1')}, {best_hps.get('filters_2')}, {best_hps.get('filters_3')}, learning rate: {best_hps.get('learning_rate')}"
)

/home/carlos/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Mejor modelo encontrado con filtros: 32, 96, 128, learning rate: 0.001


/home/carlos/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
# =============================================================================
# Entrenar el mejor modelo utilizando el conjunto de entrenamiento y validación.
# Se extrae el historial de entrenamiento para su análisis posterior.
# =============================================================================
history = best_model.fit(train_generator, validation_data=test_generator, epochs=25)

# =============================================================================
# Convertir el historial de entrenamiento a un DataFrame para analizar las métricas.
# =============================================================================
df_metrics = pd.DataFrame(history.history)

/home/carlos/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 262s 5s/step - AUC: 0.9981 - accuracy: 0.9914 - loss: 0.0419 - precision: 0.9914 - recall: 0.9914 - val_AUC: 0.6776 - val_accuracy: 0.6241 - val_loss: 1.6306 - val_precision: 0.6241 - val_recall: 0.6241
Epoch 2/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 244s 5s/step - AUC: 0.9984 - accuracy: 0.9817 - loss: 0.0501 - precision: 0.9817 - recall: 0.9817 - val_AUC: 0.9757 - val_accuracy: 0.9699 - val_loss: 0.1510 - val_precision: 0.9699 - val_recall: 0.9699
Epoch 3/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 232s 5s/step - AUC: 0.9966 - accuracy: 0.9857 - loss: 0.0586 - precision: 0.9857 - recall: 0.9857 - val_AUC: 0.9739 - val_accuracy: 0.9624 - val_loss: 0.1353 - val_precision: 0.9624 - val_recall: 0.9624
Epoch 4/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 234s 5s/step - AUC: 1.0000 - accuracy: 0.9966 - loss: 0.0216 - precision: 0.9966 - recall: 0.9966 - val_AUC: 0.9750 - val_accuracy: 0.9624 - val_loss: 0.1510 - val_precision: 0.9624 - val_recall: 0.9624
Epoch 5/25
51/51 ━━━━━━━━━━━━━━━━━━━

In [20]:
# =============================================================================
# Guardar el DataFrame con las métricas de entrenamiento en un archivo CSV,
# usando una estructura de carpetas organizada.
# =============================================================================
csv_dir = "./results/csv_files"
os.makedirs(csv_dir, exist_ok=True)

csv_metrics_path = os.path.join(csv_dir, "training_metrics_binary.csv")
df_metrics.to_csv(csv_metrics_path, index=False)
print("Métricas guardadas en:", csv_metrics_path)

Métricas guardadas en: ./results/csv_files/training_metrics_binary.csv


In [19]:
def save_model_structure(best_model, best_hps, model_type):
    """
    Guarda el modelo completo, sus pesos y los hiperparámetros en una estructura organizada.

    Args:
        best_model: Modelo de Keras a guardar.
        best_hps: Objeto de hiperparámetros (por ejemplo, de Keras Tuner).
        model_type: Cadena identificadora ("multiclass" o "binary") para crear subcarpetas específicas.
    """
    # Definir la carpeta base para el tipo de modelo (multiclass o binary)
    base_dir = f"./results/{model_type}"

    # Crear subcarpetas para modelos, pesos e hiperparámetros
    models_dir = os.path.join(base_dir, "models")
    weights_dir = os.path.join(base_dir, "weights")
    hyperparams_dir = os.path.join(base_dir, "hyperparameters")

    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(weights_dir, exist_ok=True)
    os.makedirs(hyperparams_dir, exist_ok=True)

    # Guardar el modelo completo en la carpeta 'models'
    model_path = os.path.join(models_dir, f"best_{model_type}_model.h5")
    best_model.save(model_path)
    print(f"Mejor {model_type} modelo guardado en:", model_path)

    # Guardar solo los pesos del modelo en la carpeta 'weights'
    weights_path = os.path.join(weights_dir, f"best_{model_type}.weights.h5")
    best_model.save_weights(weights_path)
    print(f"Pesos del {model_type} modelo guardados en:", weights_path)

    # Guardar los hiperparámetros del modelo en formato JSON en la carpeta 'hyperparameters'
    hyperparams_path = os.path.join(
        hyperparams_dir, f"best_{model_type}_hyperparameters.json"
    )
    with open(hyperparams_path, "w") as json_file:
        json.dump(best_hps.values, json_file, indent=4)
    print(f"Hiperparámetros del {model_type} modelo guardados en:", hyperparams_path)


# Ejemplo de uso para el modelo binario:
save_model_structure(best_model, best_hps, "binary")

Mejor binary modelo guardado en: ./results/binary/models/best_binary_model.h5
Pesos del binary modelo guardados en: ./results/binary/weights/best_binary.weights.h5
Hiperparámetros del binary modelo guardados en: ./results/binary/hyperparameters/best_binary_hyperparameters.json
